# Dropping variable with too many missing values

Note : Number of missing values in variable could be a hyper parameter

In [61]:
col_nan = train_data.isnull().sum(axis=0)

sorted_columns_values = col_nan[col_nan >= 0].sort_values()
print(sorted_columns_values)

sorted_columns = col_nan[col_nan >= 259].sort_values().index.to_list()
print(sorted_columns)
train_data = train_data.drop(sorted_columns, axis=1)
test_data = test_data.drop(sorted_columns, axis=1)

MSSubClass        0
TotalBsmtSF       0
Heating           0
SaleType          0
CentralAir        0
               ... 
FireplaceQu     690
Fence          1179
Alley          1369
MiscFeature    1406
PoolQC         1453
Length: 79, dtype: int64
['LotFrontage', 'FireplaceQu', 'Fence', 'Alley', 'MiscFeature', 'PoolQC']


# Categorical values study

we will use one-hot encoding for categorical variables to be able to move forward with dataset assessment, exploration, analysis and prediction through the use of different techniques.

categorical variables :

MSSubClass , MSZoning , MSZoning , Street , Alley , LotShape , LandContour , Utilities , LotConfig , LandSlope , Neighborhood , Condition1 , Condition2 , BldgType , HouseStyle , OverallQual , OverallCond , RoofStyle , RoofMatl , Exterior1st , Exterior2nd , MasVnrType , ExterQual , ExterCond , Foundation , BsmtQual , BsmtCond , BsmtExposure , BsmtFinType1 , BsmtFinType2 , Heating , HeatingQC , CentralAir , Electrical , KitchenQual , Functional , FireplaceQu , GarageType , GarageFinish , GarageQual , GarageCond , PavedDrive , PoolQC , Fence , MiscFeature , SaleType , SaleCondition

In [62]:
categorical_variables = [
    'MSSubClass','MSZoning','Street','LotShape',
    'LandContour','Utilities','LotConfig','LandSlope',
    'Neighborhood','Condition1','Condition2','BldgType',
    'HouseStyle','OverallQual','OverallCond','YearBuilt',
    'YearRemodAdd','RoofStyle','RoofMatl','Exterior1st',
    'Exterior2nd','MasVnrType','ExterQual','ExterCond',
    'Foundation','BsmtQual','BsmtCond','BsmtExposure',
    'BsmtFinType1','BsmtFinType2','Heating','HeatingQC',
    'CentralAir','Electrical','KitchenQual','Functional',
    'GarageType','GarageYrBlt','GarageFinish',
    'GarageQual','GarageCond','PavedDrive',
    'MoSold','YrSold','SaleType','SaleCondition'
]

quant_variables = list(set(train_data.columns.values) - set(categorical_variables))


## Categorical variables class imbalance plotting

looking at class imbalance within each categorical variable in order to drop the less represented class that do not bring any valuable information due to their rarity.

In [63]:
for cat_var in categorical_variables:
    print(train_data[cat_var].astype("category").value_counts())
    print('\n')
    

20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: MSSubClass, dtype: int64


RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64


Pave    1454
Grvl       6
Name: Street, dtype: int64


Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64


Lvl    1311
Bnk      63
HLS      50
Low      36
Name: LandContour, dtype: int64


AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64


Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: LotConfig, dtype: int64


Gtl    1382
Mod      65
Sev      13
Name: LandSlope, dtype: int64


NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
BrkSide     58
Crawfor     51
Mitchel     49
NoRidge     41
Timber      38
IDOTRR      

In [64]:
for cat_var in categorical_variables:
    print(test_data[cat_var].astype("category").value_counts())
    print('\n')

20     543
60     276
50     143
120     95
30      70
70      68
160     65
80      60
90      57
190     31
85      28
180      7
75       7
45       6
40       2
150      1
Name: MSSubClass, dtype: int64


RL         1114
RM          242
FV           74
C (all)      15
RH           10
Name: MSZoning, dtype: int64


Pave    1453
Grvl       6
Name: Street, dtype: int64


Reg    934
IR1    484
IR2     35
IR3      6
Name: LotShape, dtype: int64


Lvl    1311
HLS      70
Bnk      54
Low      24
Name: LandContour, dtype: int64


AllPub    1457
Name: Utilities, dtype: int64


Inside     1081
Corner      248
CulDSac      82
FR2          38
FR3          10
Name: LotConfig, dtype: int64


Gtl    1396
Mod      60
Sev       3
Name: LandSlope, dtype: int64


NAmes      218
OldTown    126
CollgCr    117
Somerst     96
Edwards     94
NridgHt     89
Gilbert     86
Sawyer      77
SawyerW     66
Mitchel     65
NWAmes      58
IDOTRR      56
Crawfor     52
BrkSide     50
Timber      34
NoRidge     30
S

## categorical variables one-hot encoding

In [65]:
print(quant_variables)

['FullBath', '3SsnPorch', 'MasVnrArea', 'GrLivArea', 'GarageCars', 'TotRmsAbvGrd', 'Fireplaces', 'EnclosedPorch', 'BsmtFinSF1', 'WoodDeckSF', 'HalfBath', 'BsmtFinSF2', 'GarageArea', 'LowQualFinSF', '2ndFlrSF', 'ScreenPorch', 'MiscVal', 'BsmtUnfSF', 'LotArea', 'OpenPorchSF', 'BsmtFullBath', 'TotalBsmtSF', 'BedroomAbvGr', '1stFlrSF', 'KitchenAbvGr', 'PoolArea', 'BsmtHalfBath']


In [66]:
encode_train_data = pd.DataFrame()

train_one_hot_encoded_variables = pd.get_dummies(train_data[categorical_variables], columns=categorical_variables)
test_one_hot_encoded_variables = pd.get_dummies(test_data[categorical_variables], columns=categorical_variables)

In [67]:
# Deciding the inital threshold to be 1% of dataset size
tot_instances_train = train_one_hot_encoded_variables.shape[0]
threshold = tot_instances_train*0.04

print ('The minimum count threshold is: '+str(threshold))

train_col_nan = train_one_hot_encoded_variables.sum(axis=0)
print(train_col_nan.sort_values(ascending=False))

The minimum count threshold is: 58.4
Utilities_AllPub      1459
Street_Pave           1454
Condition2_Norm       1445
RoofMatl_CompShg      1434
Heating_GasA          1428
                      ... 
ExterCond_Po             1
GarageYrBlt_1906.0       1
YearBuilt_1875           1
YearBuilt_1872           1
YearBuilt_1911           1
Length: 555, dtype: int64


In [68]:
train_threshold_columns = train_col_nan[train_col_nan <= threshold].sort_values().index.to_list()
# print(train_threshold_columns)
# dropped_one_hot_encoded_variables = train_one_hot_encoded_variables.drop(train_threshold_columns, axis=1)
absent_set = ['YearBuilt_1882','YearBuilt_1875','Exterior1st_ImStucc',
'YearBuilt_1872','GarageYrBlt_1933.0','Condition2_RRAe',
'GarageYrBlt_1906.0','GarageYrBlt_1908.0','Exterior2nd_Other',
'Heating_Floor','YearBuilt_1893','Condition2_RRAn',
'YearBuilt_1898','YearBuilt_1913','YearBuilt_1906',
'YearBuilt_1904','YearBuilt_1911','RoofMatl_ClyTile',
'RoofMatl_Membran','Utilities_NoSeWa','RoofMatl_Metal',
'RoofMatl_Roll','Electrical_Mix','Heating_OthW',
'Condition2_RRNn','YearBuilt_1885','GarageYrBlt_1914.0',
'YearBuilt_1908','YearBuilt_1892','GarageYrBlt_1929.0',
'Exterior1st_Stone','GarageQual_Ex','GarageYrBlt_1931.0',
'HouseStyle_2.5Fin']

remove_from_test = list(set(train_threshold_columns) - set(absent_set))

test_data = test_one_hot_encoded_variables.drop(remove_from_test, axis=1)

encoded_train_data = pd.merge(train_one_hot_encoded_variables, train_data[quant_variables],
                              how='outer', left_on='Id', right_on='Id')
encoded_train_data.insert(2,"SalePrice",target_train)

Id
1       208500
2       181500
3       223500
4       140000
5       250000
         ...  
1456    175000
1457    210000
1458    266500
1459    142125
1460    147500
Name: SalePrice, Length: 1460, dtype: int64

# Writing processed data back to csv file

In [69]:
encoded_train_data.to_csv("../data/processed/encoded_train_data.csv")
test_data.to_csv("../data/processed/encoded_test_data")